In [1]:
#!pip install mediapipe opencv-python==4.5.5.64
# source idea from https://www.youtube.com/watch?v=vQZ4IvB07ec&t=347s 
# source https://google.github.io/mediapipe/solutions/hands 
# полезно по оформлению https://habr.com/ru/post/485318/
# мышь https://myrusakov.ru/python-gui-automation.html


In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
# from time import time as time
import time
import math

import keyboard

import pyautogui
pyautogui.size() # Size(width=3840, height=2160)
pyautogui.PAUSE = 1
pyautogui.FAILSAFE = True


# Hands 
<img src=https://i.imgur.com/qpRACer.png />

In [3]:
# Вспомогательные фишечки
green = (0, 255, 0)
red = (0, 0, 255)
white = (255,255,255)
# cv2.putText(image, text, org, font, fontScale, color[, 
#               thickness[, lineType[, bottomLeftOrigin]]])


def text(image, text, pos=(10, 30), fontScale=1, color=red, thickness=2):
    cv2.putText(image, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 
        fontScale=fontScale, color=color, thickness=thickness)


def dist_betw_points(pt1, pt2):
    """ calc distance betw 2 points"""
    return math.sqrt((pt2[0]-pt1[0])*(pt2[0]-pt1[0]) + 
    (pt2[1]-pt1[1])*(pt2[1]-pt1[1]) + (pt2[2]-pt1[2])*(pt2[2]-pt1[2]))


def finger_connect_evt(pt1_end, pt1_prev, pt2_end, pt2_prev):
    """ connect finger event calc as a ratio of 2 distances - 
        first is the distance beetween finger ends, second - is 
        between pre-end finger points
    """ 
    # TODO!!
    pass


def pos_perst(hand, perst, coord): # нужно ли это??? хз
    return hand.landmark[perst].coord

def move_pt(pt, pt_step, mult=1):
    return ((pt[0]-pt_step[0])*mult, (pt[1]-pt_step[1])*mult) 


In [4]:
# TODO!
# DONE в первом приближении, возможно требует улучшений после привязки мыши. 
# при фронтальном ракурсе руки соотношение длин, по которому принимается решение для 
# передвижения курсора оказывается ниже порога. 

#       делать масштаб кисти для вычисления порога в зависимости от удаления от камеры.
#       тупое измерение расстояния между кончиками пальцев часто не дает эффекта, т.к. 
#       эти кончики оказываются в разных местах в зависимости от ракурса.
#       поэтомуесть идея оценить взимное расоложение пальцев с использованием других точек
#       пальца.

# делать привязку к мышке
# сделать фильтр кальмана для сглаживания трека от которого будет мышка двигаться
# делать что-то для корректной работы, если есть вторая рука в кадре
#       например, работать с двумя руками, и брать в работу только то, 

# DONE разбираться как влияет z координата пока вроде без нее работает лучше чем с ней. 
# DONE установить лимит на одну руку

# Malfunc
<img src="pics/front_palm_front_malfunc.png" />

In [7]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

камера = 0 # 0,1 - какая первая подключилась 
cap = cv2.VideoCapture(камера)
while not cap.isOpened():
    pass
# time.sleep(15)
ret, frame = cap.read()
image_hight, image_width = frame.shape[:2]
index_end = mp_hands.HandLandmark.INDEX_FINGER_TIP      # указательный конец
# index_prev = mp_hands.HandLandmark.INDEX_FINGER_DIP   # указательный 2 точка 
index_prev = mp_hands.HandLandmark.INDEX_FINGER_PIP     # указательный 2 точка
thumb_end = mp_hands.HandLandmark.THUMB_TIP             # большой конец
# thumb_prev = mp_hands.HandLandmark.THUMB_IP           # большой вторая точка
thumb_prev = mp_hands.HandLandmark.THUMB_MCP            # большой вторая точка

dist_ratio_thresh = 3.2
dist_ends, dist_prevs = 1 , 1
dist_ratio = 1
pt1_end = (100,100)
pt1_step = (100,100) # предыдущая позиция курсора для отслеживания относительного перемещения
contact = False # признак того что пальцы соединились

while cap.isOpened():
    with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8, max_num_hands =1) as hands:
        t0= time.time()
        ret, frame = cap.read()

        # Detection
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # print(f"res {results}")

        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                # print(f" {mp_hands.HAND_CONNECTIONS}")
                mp_drawing.draw_landmarks(
                    image, hand, mp_hands.HAND_CONNECTIONS)
                
                pt1_end = (hand.landmark[index_end].x, hand.landmark[index_end].y, 0)
                pt1_prev = (hand.landmark[index_prev].x, hand.landmark[index_prev].y, 0)
                
                pt2_end = (hand.landmark[thumb_end].x, hand.landmark[thumb_end].y, 0)
                pt2_prev = (hand.landmark[thumb_prev].x, hand.landmark[thumb_prev].y, 0)
                                
                dist_ends = dist_betw_points(pt1_end, pt2_end)
                dist_ends = dist_ends if dist_ends !=0 else 0.01 # prevent div by zero
                
                dist_prevs = dist_betw_points(pt1_prev, pt2_prev)

        contact = True if dist_ratio > dist_ratio_thresh else False

        # вычислить соотношение дистанций соответствующих точек пальцев
        dist_ratio = dist_prevs/dist_ends
        # вычислить перемещение мыши 
        if contact:
            dp = move_pt(pt1_end, pt1_step, mult=5000)# сразу умножим движ на mult
        # переписать значение мыши предыдущего цикла
        pt1_step = pt1_end
        # привязать мышь к концу указательного пальца
        pyautogui.moveRel(dp[0], dp[1], _pause=False)        
        
        time_spend = time.time() - t0
        
        color = green if contact else False
        text(image, f'{dist_ends:.2f} {dist_prevs:.2f} {dist_ratio:.2f}', color=color)
        text(image, f'{pt1_end[0]:.2f} {pt1_end[1]:.2f} : {dp[0]:.0f} {dp[1]:.0f} ', 
                    pos=(300, 30), color=white, thickness=1)
        text(image, f'ts {time_spend:.3}', pos=(0, 60), color = white)
        # assert dp[0]*100<10, dp[0]

        image = cv2.resize(image, (image.shape[1]*3,image.shape[0]*3)) # optional
        cv2.imshow('hands', image)
        if cv2.waitKey(1) == 27:
            break
cap.release()
cv2.destroyAllWindows()


In [16]:
results.multi_hand_landmarks??
# mp_hands.HAND_CONNECTIONS??
# mp_drawing.DrawingSpec??


Type:        NoneType
String form: None
Docstring:   <no docstring>


In [ ]:
# гоняем мыша
# !pip install pyautogui

In [1]:
import pyautogui
pyautogui.size() # Size(width=3840, height=2160)
pyautogui.PAUSE = 1
pyautogui.FAILSAFE = True


In [19]:
# d1 = {x:100; y:200}
duration = 0
for i in range(10):
    # pyautogui.moveRel(100, 0, duration=duration)
    pyautogui.moveRel(100, 0)
    pyautogui.moveRel(0, 100, duration=duration)
    pyautogui.moveRel(-100, 0, duration=duration)
    pyautogui.moveRel(0, -100, duration=duration)

In [ ]:
import keyboard, time

while True:
    if keyboard.is_pressed('ctrl'):
        time.sleep(0.1)
        print(f"ctrl pressed")
        keyboard.release("ctrl")
        # keyboard.send("ctrl")
        # keyboard.unhook_all()
        # break


In [18]:
a=(1,2)
b=(3,4)
b[0]-a[0], b[1]-a[1]
# b-a так нельзя
a*b[0] # a три раза подряд


(1, 2, 1, 2, 1, 2)